In [1]:
# Загрузим все необходимое - библиотеку, исходные данные

import pandas as pd

users_df = pd.read_csv('./data/users.csv', encoding='cp1251', delimiter=';')
newusers_df = pd.read_csv('./data/newusers.csv', encoding='cp1251', delimiter=';')
folders_df = pd.read_csv('./data/folders.csv', encoding='cp1251', delimiter=';')
newfolders_df = pd.read_csv('./data/newfolders.csv', encoding='cp1251', delimiter=';')
folders_content_df = pd.read_csv('./data/folders_content.csv', encoding='cp1251', delimiter=';')
newfolders_content_df = pd.read_csv('./data/newfolders_content.csv', encoding='cp1251', delimiter=';')

In [2]:
# Посмотрим, что есть внутри полученных наборов данных

In [3]:
users_df

,id,name
0,1,1013369
1,2,893891
2,3,897313
3,4,965405
4,5,984513
5,6,996005


In [4]:
newusers_df

,id,name
0,1,1013369
1,2,893891
2,3,897313
3,4,965405
4,5,984513
5,6,996005


In [5]:
# С пользователями вроде пока все понятно - здесь миграция не привнесла изменений

In [6]:
folders_df

,id,sid,name,type,owner,parent,tm
0,1531382,NaN,NaN,1,1013369,0,1585121402
1,1531384,0,Документы на контроле,2,1013369,1531382,1584952099
2,1531383,DAFBE4A59A5FEEE3888E180E15D9DF7D,Мои папки,1,1013369,1531382,1585121402
3,1532656,FB3B19B0FB89ADE9633B68D2DD839A8E,судебка по признанию сделки недействительной,32,1013369,1531383,1585121402
4,756032,NaN,NaN,1,893891,0,1566896684
...,...,...,...,...,...,...,...
214,1408802,55A325F01A4F6B31BA3B26812A90F71D,ЧЗБХ,32,996005,1406465,1573111401
215,1537427,D5D84DCCB24E87EEF30733D1046F3B53,Коронавирус-Кризис,32,996005,1406465,1591797711
216,1525829,DB0DAD86B40278F14FA50DD2ED2AFB76,КЖ ФТС-Сип,32,996005,1406465,1583834895
217,1408732,F21C81D73F293CF74758334647F70DF8,ДДУ,32,996005,1406465,1582538124


In [7]:
newfolders_df

,id,sid,name,type,owner,parent,tm
0,942643,041D4584B42D2A9EAD449494F1864F70,Саша,32,897313,767068,1526742953
1,1189505,74484AC5DB2928496880A7E6E6C1CFEF,корпорации,32,893891,756033,1544353712
2,1029514,0349B41A7AD1D5CA8A918DEE8B96E786,Изменения с 2019,32,897313,767068,1540588093
3,952206,F854195F0F62F66F626C76D5229871D2,иск о признании,32,893891,756033,1529470364
4,1181131,DD652660A2E3F997BFF72B1714EF2F76,НТЭК,33,893891,756033,1543335375
...,...,...,...,...,...,...,...
218,1525829,DB0DAD86B40278F14FA50DD2ED2AFB76,КЖ ФТС-Сип,32,996005,1406465,1583834895
219,1408732,F21C81D73F293CF74758334647F70DF8,ДДУ,32,996005,1406465,1582538124
220,1521463,FD1F96EDBD3AFC4AA5487166F30FB2B5,Адрес ответчика неизвестен,32,996005,1406465,1583140509
221,1406464,NaN,NaN,1,996005,0,1591797711


In [8]:
# Сразу видно, что после миграции количество папок увеличилось на 4.
# Найдем все различия между папками по полям 'id', 'name', 'type', 'owner', 'parent'

In [9]:
# OUTER JOIN. Исходим из того, что алгоритм миграции не должен менять поля 'id', 'name', 'type', 'owner', 'parent'.

# Получили набор данных с отличиями/сходствами между старыми и новыми папками пользователей.
# Для различения создан столбец '_merge', где значения:
# _merge == 'both', если пункт есть в обеих (и новых, и старых) папках
# _merge == 'left_only', если папка есть только в старой таблице
# _merge == 'right_only', если папка появилась только после миграции

df = folders_df.merge(newfolders_df, how='outer', on=['id', 'name', 'type', 'owner', 'parent'], 
              suffixes=['', '_new'], indicator=True)

In [10]:
# Посмотрим только различия между папками

df[df._merge != 'both']

,id,sid,name,type,owner,parent,tm,sid_new,tm_new,_merge
59,1610881,167B49C9FD6AFD5F68FB44EDAB8590AC,Новая папка,32,965405,1243370,1.592567e+09,NaN,NaN,left_only
75,1540906,24F6F6D85EFF60D105456829AF42A328,закупки,32,965405,1243370,1.587068e+09,NaN,NaN,left_only
141,1537347,A5675FE3614951B5F4E43E73F5E3BFF2,_вирус общественное питание,32,965405,1243370,1.592661e+09,NaN,NaN,left_only
219,1621168,NaN,_коммуналка потребители,32,965405,1243370,NaN,10C37BC67C52678385CDF3EDF9826949,1.597524e+09,right_only
220,1625573,NaN,ТП-отпуск,32,965405,1243370,NaN,5411820DBC2BCB387FD41597537C829C,1.596441e+09,right_only
221,1613634,NaN,заем-кредит,32,965405,1243370,NaN,9AD76028B5036C8A2C4CC547BAE7C4D1,1.593506e+09,right_only
222,1629882,NaN,ограничение ответственности,32,965405,1243370,NaN,A774CD70009CB7F55B3F2FF069B4454D,1.597518e+09,right_only
223,1623185,NaN,ускорение рассмотрения дела,32,965405,1243370,NaN,ABDCDF959737321E9D2BDBBD1CF72D1F,1.595601e+09,right_only
224,1613635,NaN,банкротство,32,965405,1243370,NaN,FA91E2D4F4A2756A0F0BD922596021A4,1.593506e+09,right_only
225,1631113,NaN,Фитнес для инвалидов,32,965405,1243370,NaN,FEC9AD9899BB8D49039508BCE2992315,1.597913e+09,right_only


In [11]:
# Большинство папок перенеслось успешно - их 216:

df[df._merge == 'both']

,id,sid,name,type,owner,parent,tm,sid_new,tm_new,_merge
0,1531382,NaN,NaN,1,1013369,0,1.585121e+09,NaN,1.585121e+09,both
1,1531384,0,Документы на контроле,2,1013369,1531382,1.584952e+09,0,1.584952e+09,both
2,1531383,DAFBE4A59A5FEEE3888E180E15D9DF7D,Мои папки,1,1013369,1531382,1.585121e+09,DAFBE4A59A5FEEE3888E180E15D9DF7D,1.585121e+09,both
3,1532656,FB3B19B0FB89ADE9633B68D2DD839A8E,судебка по признанию сделки недействительной,32,1013369,1531383,1.585121e+09,FB3B19B0FB89ADE9633B68D2DD839A8E,1.585121e+09,both
4,756032,NaN,NaN,1,893891,0,1.566897e+09,NaN,1.566897e+09,both
...,...,...,...,...,...,...,...,...,...,...
214,1408802,55A325F01A4F6B31BA3B26812A90F71D,ЧЗБХ,32,996005,1406465,1.573111e+09,55A325F01A4F6B31BA3B26812A90F71D,1.573111e+09,both
215,1537427,D5D84DCCB24E87EEF30733D1046F3B53,Коронавирус-Кризис,32,996005,1406465,1.591798e+09,D5D84DCCB24E87EEF30733D1046F3B53,1.591798e+09,both
216,1525829,DB0DAD86B40278F14FA50DD2ED2AFB76,КЖ ФТС-Сип,32,996005,1406465,1.583835e+09,DB0DAD86B40278F14FA50DD2ED2AFB76,1.583835e+09,both
217,1408732,F21C81D73F293CF74758334647F70DF8,ДДУ,32,996005,1406465,1.582538e+09,F21C81D73F293CF74758334647F70DF8,1.582538e+09,both


In [12]:
# id пропавших папок

left_only_np = df[df._merge == 'left_only']['id'].values
left_only_np

array([1610881, 1540906, 1537347])

In [13]:
# список пропавших папок

folders_df[folders_df.id.isin(left_only_np)]

,id,sid,name,type,owner,parent,tm
59,1610881,167B49C9FD6AFD5F68FB44EDAB8590AC,Новая папка,32,965405,1243370,1592567460
75,1540906,24F6F6D85EFF60D105456829AF42A328,закупки,32,965405,1243370,1587068238
141,1537347,A5675FE3614951B5F4E43E73F5E3BFF2,_вирус общественное питание,32,965405,1243370,1592660579


In [14]:
# для самопроверки - папок с такими id-номерами в новой таблице не существуют

newfolders_df[newfolders_df.id.isin(left_only_np)]

,id,sid,name,type,owner,parent,tm


In [15]:
# id появившихся после миграции папок

right_only_np = df[df._merge == 'right_only']['id'].values
right_only_np

array([1621168, 1625573, 1613634, 1629882, 1623185, 1613635, 1631113])

In [16]:
# список появившихся после миграции папок

newfolders_df[newfolders_df.id.isin(right_only_np)]

,id,sid,name,type,owner,parent,tm
61,1621168,10C37BC67C52678385CDF3EDF9826949,_коммуналка потребители,32,965405,1243370,1597523781
95,1625573,5411820DBC2BCB387FD41597537C829C,ТП-отпуск,32,965405,1243370,1596441195
135,1613634,9AD76028B5036C8A2C4CC547BAE7C4D1,заем-кредит,32,965405,1243370,1593506082
144,1629882,A774CD70009CB7F55B3F2FF069B4454D,ограничение ответственности,32,965405,1243370,1597517856
146,1623185,ABDCDF959737321E9D2BDBBD1CF72D1F,ускорение рассмотрения дела,32,965405,1243370,1595600522
161,1613635,FA91E2D4F4A2756A0F0BD922596021A4,банкротство,32,965405,1243370,1593506454
187,1631113,FEC9AD9899BB8D49039508BCE2992315,Фитнес для инвалидов,32,965405,1243370,1597912677


In [17]:
# для самопроверки - с такими id-номерами в старых папках ничего не было

folders_df[folders_df.id.isin(right_only_np)]

,id,sid,name,type,owner,parent,tm


# Промежуточный вывод:

In [18]:
# Предположили, что мигрировать должны все папки всех пользователей без изменений. Если находим какие-то
# Изменения - для нас это тревожный сигнал - или потеря данных, или откуда-то пришли лишние данные.

# Обнаружена потеря данных при миграции, пропали папки:

folders_df[folders_df.id.isin(left_only_np)]

,id,sid,name,type,owner,parent,tm
59,1610881,167B49C9FD6AFD5F68FB44EDAB8590AC,Новая папка,32,965405,1243370,1592567460
75,1540906,24F6F6D85EFF60D105456829AF42A328,закупки,32,965405,1243370,1587068238
141,1537347,A5675FE3614951B5F4E43E73F5E3BFF2,_вирус общественное питание,32,965405,1243370,1592660579


In [19]:
# Откуда-то взялись лишние папки:

newfolders_df[newfolders_df.id.isin(right_only_np)]

,id,sid,name,type,owner,parent,tm
61,1621168,10C37BC67C52678385CDF3EDF9826949,_коммуналка потребители,32,965405,1243370,1597523781
95,1625573,5411820DBC2BCB387FD41597537C829C,ТП-отпуск,32,965405,1243370,1596441195
135,1613634,9AD76028B5036C8A2C4CC547BAE7C4D1,заем-кредит,32,965405,1243370,1593506082
144,1629882,A774CD70009CB7F55B3F2FF069B4454D,ограничение ответственности,32,965405,1243370,1597517856
146,1623185,ABDCDF959737321E9D2BDBBD1CF72D1F,ускорение рассмотрения дела,32,965405,1243370,1595600522
161,1613635,FA91E2D4F4A2756A0F0BD922596021A4,банкротство,32,965405,1243370,1593506454
187,1631113,FEC9AD9899BB8D49039508BCE2992315,Фитнес для инвалидов,32,965405,1243370,1597912677


In [20]:
# Разница между количеством лишних и недостающих папок дает 4, что как раз равно разнице строк таблиц

newfolders_df[newfolders_df.id.isin(right_only_np)]['id'].count() - \
folders_df[folders_df.id.isin(left_only_np)]['id'].count()

4

# Разберемся с таблицами folders_content и newfolders_content

In [21]:
newfolders_content_df

,id,objid,docn,basename,dtmod,sliceid,family,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,9666571,790306,92632,CJI,0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
1,9666572,895356,109485,CJI,0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
2,9666574,904125,490932,ARB001,0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
3,9666616,789324,513568,ARB001,0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
4,10225455,790305,96569,CJI,0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,51960443,1621168,257459,PBI,0,32776,37,NaN,NaN,NaN,NaN,NaN,NaN
2147,52173115,1631113,359011,RZR,0,32913,0,NaN,NaN,NaN,NaN,NaN,NaN
2148,52173116,1631113,113396,CJI,0,32798,32798,NaN,NaN,NaN,NaN,NaN,NaN
2149,52709608,1243371,349468,RZR,56548,32913,0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# В файле с таблицей newfolders_content.csv много лишних разделителей, это привело к появлению пустых столбцов в
# загруженном наборе данных newfolders_content_df
# Столбец с False содержит хотя бы одно значение
# Столбец с True полностью пустой

newfolders_content_df.isna().all(axis=0)

id             False
objid          False
docn           False
basename       False
dtmod          False
sliceid        False
family         False
Unnamed: 7      True
Unnamed: 8      True
Unnamed: 9      True
Unnamed: 10     True
Unnamed: 11     True
Unnamed: 12     True
dtype: bool

In [23]:
# Удалим пустые столбцы

newfolders_content_df = newfolders_content_df.drop(['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', \
                                                    'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], axis=1)
newfolders_content_df

,id,objid,docn,basename,dtmod,sliceid,family
0,9666571,790306,92632,CJI,0,-1,-1
1,9666572,895356,109485,CJI,0,-1,-1
2,9666574,904125,490932,ARB001,0,-1,-1
3,9666616,789324,513568,ARB001,0,-1,-1
4,10225455,790305,96569,CJI,0,-1,-1
...,...,...,...,...,...,...,...
2146,51960443,1621168,257459,PBI,0,32776,37
2147,52173115,1631113,359011,RZR,0,32913,0
2148,52173116,1631113,113396,CJI,0,32798,32798
2149,52709608,1243371,349468,RZR,56548,32913,0


In [24]:
# Набор данных, полученный из folders_content.csv выглядит готовым к работе.

folders_content_df

,id,objid,docn,basename,dtmod,sliceid,family
0,9666571,790306,92632,CJI,0,-1,-1
1,9666572,895356,109485,CJI,0,-1,-1
2,9666574,904125,490932,ARB001,0,-1,-1
3,9666576,848854,5155,CJI,0,-1,-1
4,9666577,848854,7242,CJI,0,-1,-1
...,...,...,...,...,...,...,...
2056,48952762,1537346,127572,CJI,25708,32798,32798
2057,48952763,1537346,89989,PAPB,34582,32938,3
2058,48952764,1537346,100118,PAPB,0,32938,3
2059,48952765,1537346,195178,QSA,64164,QSA002,2


In [25]:
# Документы из старых папок, не привязанные ни к каким ИБ

folders_content_df[folders_content_df.basename.isna()]

,id,objid,docn,basename,dtmod,sliceid,family
1681,43585291,789502,544324,NaN,2,NaN,4


In [26]:
# Документы из папок после миграции, не привязанные ни к каким ИБ

newfolders_content_df[newfolders_content_df.basename.isna()]

,id,objid,docn,basename,dtmod,sliceid,family
345,26451587,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,NaN,0,NaN,65535
348,26451590,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,NaN,57631,NaN,65535
350,26451592,1366763,1069378,NaN,26073,NaN,65535
351,26451593,1366763,1071812,NaN,0,NaN,65535
352,26451594,1366763,1181318,NaN,56446,NaN,65535
353,26451595,1366763,1183436,NaN,2498,NaN,65535
354,26451596,1366763,1225098,NaN,0,NaN,65535
1037,35169800,1468481,1668105,NaN,57571,NaN,65535


In [27]:
# 17 документов не привязаны ни к одному из ИБ после миграции.

newfolders_content_df[newfolders_content_df.basename.isna()].id.values

array([26451587, 26451588, 26451590, 26451591, 26451592, 26451593,
       26451594, 26451595, 26451596, 35169800, 35169801, 35169802,
       35169803, 35169804, 35169805, 35169806, 43585291])

In [28]:
# Хотя возможно документу не обязательно быть привязанным к какой-либо информационной базе.

# Опять же видна разница в количестве строк - механизм миграции привел к видимым изменениям в содержимом папок.
# Как и в случае таблиц с папками найдем отличия в полученных таблицах

In [29]:
# INNER JOIN. Возьмем только записи с id, которые есть в folders_content и newfolders_content одновременно

df = pd.merge(folders_content_df, newfolders_content_df, how='inner', on='id')
df

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
0,9666571,790306,92632,CJI,0,-1,-1,790306,92632,CJI,0,-1,-1
1,9666572,895356,109485,CJI,0,-1,-1,895356,109485,CJI,0,-1,-1
2,9666574,904125,490932,ARB001,0,-1,-1,904125,490932,ARB001,0,-1,-1
3,9666616,789324,513568,ARB001,0,-1,-1,789324,513568,ARB001,0,-1,-1
4,10225455,790305,96569,CJI,0,-1,-1,790305,96569,CJI,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,48905263,1253309,283377,AMS,10,24,18,1253309,283377,AMS,10,24,18
1734,48905264,1253309,516265,ARB,36802,ARB002,4,1253309,516265,ARB,36802,ARB002,4
1735,48905265,1253309,561090,ARB,48126,ARB002,4,1253309,561090,ARB,48126,ARB002,4
1736,48905266,1253309,572414,ARB,13962,ARB002,4,1253309,572414,ARB,13962,ARB002,4


In [30]:
# 67 записей с изменившимся полем 'basename'

df.query('basename_x != basename_y')

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
345,26451587,1366763,863872,MARB,40579,MARB009,41,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,MARB,18015,MARB009,41,1366763,958083,NaN,0,NaN,65535
347,26451589,1366763,1001055,MARB,30822,MARB009,41,1366763,1001055,AMS,85,24,18
348,26451590,1366763,1013862,MARB,6806,MARB009,41,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,MARB,20802,MARB009,41,1366763,1055382,NaN,57631,NaN,65535
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,35169803,1468481,1677634,MARB,49514,MARB009,41,1468481,1677634,NaN,0,NaN,65535
1041,35169804,1468481,1687914,MARB,50215,MARB009,41,1468481,1687914,NaN,56446,NaN,65535
1042,35169805,1468481,1688615,MARB,52580,MARB009,41,1468481,1688615,NaN,2498,NaN,65535
1043,35169806,1468481,1690980,MARB,1855,MARB009,41,1468481,1690980,NaN,0,NaN,65535


In [31]:
# список правильно отработавших миграций документов из MARB в MAPS

temp_df = df.query('basename_x != basename_y')[df.query('basename_x != basename_y').basename_y == 'MAPS ']
temp_df

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
988,35169751,1468481,693157,MARB,42244,MARB009,41,1468481,693157,MAPS,42244,MARB009,41
989,35169752,1468481,763140,MARB,63498,MARB009,41,1468481,763140,MAPS,63498,MARB009,41
990,35169753,1468481,784394,MARB,62986,MARB009,41,1468481,784394,MAPS,62986,MARB009,41
991,35169754,1468481,849418,MARB,65065,MARB009,41,1468481,849418,MAPS,65065,MARB009,41
992,35169755,1468481,851497,MARB,3573,MARB009,41,1468481,851497,MAPS,3573,MARB009,41
993,35169756,1468481,855541,MARB,24866,MARB009,41,1468481,855541,MAPS,24866,MARB009,41
994,35169757,1468481,876834,MARB,35764,MARB009,41,1468481,876834,MAPS,35764,MARB009,41
995,35169758,1468481,887732,MARB,36071,MARB009,41,1468481,887732,MAPS,36071,MARB009,41
996,35169759,1468481,888039,MARB,64943,MARB009,41,1468481,888039,MAPS,64943,MARB009,41
997,35169760,1468481,916911,MARB,29705,MARB009,41,1468481,916911,MAPS,29705,MARB009,41


In [32]:
# В отношении 49 записей алгоритм миграции отработал правильно

temp_df.shape[0]

49

In [33]:
# Не правильно отработавшие миграции из MARB в MAPS собраны здесь, плюс одно поле с NaN, т.к. питон видит
# значения NaN не равными самому себе

temp_df = df.query('basename_x != basename_y')[(df.query('basename_x != basename_y').basename_y != 'MAPS ')]
temp_df

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
345,26451587,1366763,863872,MARB,40579,MARB009,41,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,MARB,18015,MARB009,41,1366763,958083,NaN,0,NaN,65535
347,26451589,1366763,1001055,MARB,30822,MARB009,41,1366763,1001055,AMS,85,24,18
348,26451590,1366763,1013862,MARB,6806,MARB009,41,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,MARB,20802,MARB009,41,1366763,1055382,NaN,57631,NaN,65535
350,26451592,1366763,1069378,MARB,23236,MARB009,41,1366763,1069378,NaN,26073,NaN,65535
351,26451593,1366763,1071812,MARB,1670,MARB009,41,1366763,1071812,NaN,0,NaN,65535
352,26451594,1366763,1181318,MARB,3788,MARB009,41,1366763,1181318,NaN,56446,NaN,65535
353,26451595,1366763,1183436,MARB,45450,MARB009,41,1366763,1183436,NaN,2498,NaN,65535
354,26451596,1366763,1225098,MARB,5766,MARB009,41,1366763,1225098,NaN,0,NaN,65535


In [34]:
# Вычтем единицу, т.к. попало значение с NaN в сравниваемых полях 'basename'.
# Это количество не корректно отработавших миграций документов из ИБ MARB в MAPS

temp_df.shape[0] - 1

17

In [35]:
# Алгоритм миграции не внес изменений в поле 'objid' 

df.query('objid_x != objid_y')

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y


In [36]:
# Алгоритм миграции не внес изменений в поле 'docn' 

df.query('docn_x != docn_y')

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y


In [37]:
# Алгоритм миграции изменил поле 'sliceid' в 17 случаях

df.query('sliceid_x != sliceid_y')

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
345,26451587,1366763,863872,MARB,40579,MARB009,41,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,MARB,18015,MARB009,41,1366763,958083,NaN,0,NaN,65535
347,26451589,1366763,1001055,MARB,30822,MARB009,41,1366763,1001055,AMS,85,24,18
348,26451590,1366763,1013862,MARB,6806,MARB009,41,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,MARB,20802,MARB009,41,1366763,1055382,NaN,57631,NaN,65535
350,26451592,1366763,1069378,MARB,23236,MARB009,41,1366763,1069378,NaN,26073,NaN,65535
351,26451593,1366763,1071812,MARB,1670,MARB009,41,1366763,1071812,NaN,0,NaN,65535
352,26451594,1366763,1181318,MARB,3788,MARB009,41,1366763,1181318,NaN,56446,NaN,65535
353,26451595,1366763,1183436,MARB,45450,MARB009,41,1366763,1183436,NaN,2498,NaN,65535
354,26451596,1366763,1225098,MARB,5766,MARB009,41,1366763,1225098,NaN,0,NaN,65535


In [38]:
# В скольких записях изменилось поле 'sliceid', минус запись со значением NaN

df.query('sliceid_x != sliceid_y').shape[0] - 1

17

In [39]:
# Алгоритм миграции изменил поле 'family' в 17 случаях

df.query('family_x != family_y')

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
345,26451587,1366763,863872,MARB,40579,MARB009,41,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,MARB,18015,MARB009,41,1366763,958083,NaN,0,NaN,65535
347,26451589,1366763,1001055,MARB,30822,MARB009,41,1366763,1001055,AMS,85,24,18
348,26451590,1366763,1013862,MARB,6806,MARB009,41,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,MARB,20802,MARB009,41,1366763,1055382,NaN,57631,NaN,65535
350,26451592,1366763,1069378,MARB,23236,MARB009,41,1366763,1069378,NaN,26073,NaN,65535
351,26451593,1366763,1071812,MARB,1670,MARB009,41,1366763,1071812,NaN,0,NaN,65535
352,26451594,1366763,1181318,MARB,3788,MARB009,41,1366763,1181318,NaN,56446,NaN,65535
353,26451595,1366763,1183436,MARB,45450,MARB009,41,1366763,1183436,NaN,2498,NaN,65535
354,26451596,1366763,1225098,MARB,5766,MARB009,41,1366763,1225098,NaN,0,NaN,65535


In [40]:
df.query('family_x != family_y').shape[0]

17

In [41]:
# Вынес еще раз изменения поля 'basename', которые я посчитал не соответствующими ожидаемым.
# Они совпали с изменениями в полях 'sliceid' и 'family', кроме записи с id=43585291, оно вошло из-за значения NaN.
# Возможно, алгоритм миграции как-то связанно вносит изменения в эти поля.

df.query('basename_x != basename_y')[(df.query('basename_x != basename_y').basename_y != 'MAPS ')]

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
345,26451587,1366763,863872,MARB,40579,MARB009,41,1366763,863872,NaN,2,NaN,65535
346,26451588,1366763,958083,MARB,18015,MARB009,41,1366763,958083,NaN,0,NaN,65535
347,26451589,1366763,1001055,MARB,30822,MARB009,41,1366763,1001055,AMS,85,24,18
348,26451590,1366763,1013862,MARB,6806,MARB009,41,1366763,1013862,NaN,57571,NaN,65535
349,26451591,1366763,1055382,MARB,20802,MARB009,41,1366763,1055382,NaN,57631,NaN,65535
350,26451592,1366763,1069378,MARB,23236,MARB009,41,1366763,1069378,NaN,26073,NaN,65535
351,26451593,1366763,1071812,MARB,1670,MARB009,41,1366763,1071812,NaN,0,NaN,65535
352,26451594,1366763,1181318,MARB,3788,MARB009,41,1366763,1181318,NaN,56446,NaN,65535
353,26451595,1366763,1183436,MARB,45450,MARB009,41,1366763,1183436,NaN,2498,NaN,65535
354,26451596,1366763,1225098,MARB,5766,MARB009,41,1366763,1225098,NaN,0,NaN,65535


In [42]:
# В качестве итога подсчитаем количество записей с ожидаемыми значениями.
# Набор данных df, который получен путем INNER-соединения таблиц 'folders_content' и 'newfolders_content'
# Из него нужно вычесть набор данных, в котором собраны все изменения поля 'family', т.к. все другие наборы данных
# совпадают с ним (за исключением строки с NaN-полями, ее тоже добавим в ожидаемые)

# Получилось 1721 записей

df.drop(df.query('family_x != family_y').index)

,id,objid_x,docn_x,basename_x,dtmod_x,sliceid_x,family_x,objid_y,docn_y,basename_y,dtmod_y,sliceid_y,family_y
0,9666571,790306,92632,CJI,0,-1,-1,790306,92632,CJI,0,-1,-1
1,9666572,895356,109485,CJI,0,-1,-1,895356,109485,CJI,0,-1,-1
2,9666574,904125,490932,ARB001,0,-1,-1,904125,490932,ARB001,0,-1,-1
3,9666616,789324,513568,ARB001,0,-1,-1,789324,513568,ARB001,0,-1,-1
4,10225455,790305,96569,CJI,0,-1,-1,790305,96569,CJI,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,48905263,1253309,283377,AMS,10,24,18,1253309,283377,AMS,10,24,18
1734,48905264,1253309,516265,ARB,36802,ARB002,4,1253309,516265,ARB,36802,ARB002,4
1735,48905265,1253309,561090,ARB,48126,ARB002,4,1253309,561090,ARB,48126,ARB002,4
1736,48905266,1253309,572414,ARB,13962,ARB002,4,1253309,572414,ARB,13962,ARB002,4


In [43]:
# Документы, id которых есть в folders_content, но нет в df, т.е. пропавших при миграции - их 323

folders_content_df[~folders_content_df.id.isin(df.id)]

,id,objid,docn,basename,dtmod,sliceid,family
3,9666576,848854,5155,CJI,0,-1,-1
4,9666577,848854,7242,CJI,0,-1,-1
5,9666578,848854,7250,CJI,0,-1,-1
6,9666579,848854,8485,CJI,0,-1,-1
7,9666580,848854,9019,CJI,0,-1,-1
...,...,...,...,...,...,...,...
2056,48952762,1537346,127572,CJI,25708,32798,32798
2057,48952763,1537346,89989,PAPB,34582,32938,3
2058,48952764,1537346,100118,PAPB,0,32938,3
2059,48952765,1537346,195178,QSA,64164,QSA002,2


In [44]:
# Документы, id которых есть в newfolders_content, но нет df, т.е. появившиеся при миграции - их 413

newfolders_content_df[~newfolders_content_df.id.isin(df.id)]

,id,objid,docn,basename,dtmod,sliceid,family
1738,49107712,1613634,18780,CMB,0,32799,32799
1739,49107979,1613635,18520,CMB,1,32799,32799
1740,49900999,1243404,545248,ARB,0,ARB002,4
1741,49901000,1243404,388314,MARB,18100,RAMSMARB,41
1742,49901001,1243404,411316,MARB,19545,RAMSMARB,41
...,...,...,...,...,...,...,...
2146,51960443,1621168,257459,PBI,0,32776,37
2147,52173115,1631113,359011,RZR,0,32913,0
2148,52173116,1631113,113396,CJI,0,32798,32798
2149,52709608,1243371,349468,RZR,56548,32913,0
